Before you turn in your homework, make sure everything runs as expected.

Make sure you execute every single code cell.

**IMPORTANT:**

* **DO NOT DELETE ANY CELL** and do not change the title of the Notebook.

* Before you turn in your homework, make sure everything runs as expected: restart the kernel (in the menubar, select Kernel $\rightarrow$ Restart) and then run all cells (in the menubar, select Cell $\rightarrow$ Run All).

Fill your name below:

In [1]:
name = "Mark Yamakaitis"

# Two-dimensional lid-driven cavity flow

For the take-home part of the final exam, you will solve the two-dimensional Navier-Stokes equations for an incompressible flow in a square cavity.

The non-dimensional Navier-Stokes equation in vector form:

$$
\begin{equation}
    \frac{\partial \mathbf{u}}{\partial t} + \left( \mathbf{u} \cdot \nabla \right) \mathbf{u} = - \nabla p + \frac{1}{Re} \nabla^2 \mathbf{u}
\end{equation}
$$

where $\mathbf{u}$ is the velocity vector field, $p$ is the pressure scalar field, and $Re$ is the Reynolds number.

The equation above represents three scalar equations, one for each velocity component.
But you will solve it in two dimensions, so there will be two scalar equations.

The continuity equation for an incompressible flow is:

$$
\begin{equation}
    \nabla \cdot \mathbf{u} = 0
\end{equation}
$$

It says that the velocity field is divergence-free.

You will use an explicit scheme to discretize the equations in time:

$$
\begin{equation}
    \frac{\mathbf{u}^{n+1} - \mathbf{u}^n}{\Delta t} = -\nabla p - \left[ \left( \mathbf{u} \cdot \nabla \right) \mathbf{u} \right]^n + \frac{1}{Re} \left[ \nabla^2 \mathbf{u} \right]^n
\end{equation}
$$

### Fractional-step method

To solve the equations, you will use a fractional-step method.
The essence of the method is to first solve an equation for an intermediate velocity field that does not satisfy the continuity equation.
In a second step, the intermediate velocity is projected onto the space of divergence-free fields.

First, we solve an equation for an intermediate velocity field $\mathbf{u}^\star$ is:

$$
\begin{equation}
    \frac{\mathbf{u}^\star - \mathbf{u}^n}{\Delta t} = - \left[ \left( \mathbf{u} \cdot \nabla \right) \mathbf{u} \right]^n + \frac{1}{Re} \left[ \nabla^2 \mathbf{u} \right]^n
\end{equation}
$$

(It corresponds to the discrete Navier-Stokes equation without the pressure-gradient term.)

The projection-step equation is:

$$
\begin{equation}
    \frac{\mathbf{u}^{n+1} - \mathbf{u}^\star}{\Delta t} = -\nabla p
\end{equation}
$$

which, after re-arranging the terms, is equivalent to:

$$
\begin{equation}
    \mathbf{u}^{n+1} = \mathbf{u}^\star - \Delta t \; \nabla p
\end{equation}
$$

Calculating $\mathbf{u}^{n+1}$ requires us to first get the pressure field $p$.
We can derive an equation for the pressure by taking the divergence on both sides of the projection equation:

$$
\begin{equation}
    \nabla \cdot \mathbf{u}^{n+1} = \nabla \cdot \mathbf{u}^\star - \Delta t \; \nabla^2 p
\end{equation}
$$

Because we require the final velocity to have no divergence, we get:

$$
\begin{equation}
    \nabla^2 p = \frac{1}{\Delta t} \nabla \cdot \mathbf{u}^\star
\end{equation}
$$

This is our pressure Poisson equation.

You will discretize the spatial operators using central-difference formula.
As a reminder:

$$
\begin{equation}
    \frac{\partial f}{\partial x}\Big|_{i, j} = \frac{f_{i, j+1} - f_{i, j-1}}{2 \Delta x} + \mathcal{O} \left( \Delta x^2 \right)
\end{equation}
$$

$$
\begin{equation}
    \frac{\partial f}{\partial y}\Big|_{i, j} = \frac{f_{i+1, j} - f_{i-1, j}}{2 \Delta y} + \mathcal{O} \left( \Delta y^2 \right)
\end{equation}
$$

$$
\begin{equation}
    \frac{\partial^2 f}{\partial x^2}\Big|_{i,j} = \frac{f_{i, j-1} - 2 f_{i, j} + f_{i, j+1}}{\Delta x^2} + \mathcal{O} \left( \Delta x^2 \right)
\end{equation}
$$

$$
\begin{equation}
    \frac{\partial^2 f}{\partial y^2}\Big|_{i,j} = \frac{f_{i-1, j} - 2 f_{i, j} + f_{i+1, j}}{\Delta y^2} + \mathcal{O} \left( \Delta y^2 \right)
\end{equation}
$$

In summary, each time step you should:

1. Solve the two-dimensional discrete Navier-Stokes equation without the pressure-gradient term for the intermediate velocity field $\mathbf{u}^\star$.

$$
\begin{eqnarray}
    \frac{u_{ij}^\star - u_{ij}^n}{\Delta t} = - u_{ij}^n \frac{u_{i, j+1}^n - u_{i, j-1}^n}{2 \Delta x} - v_{ij}^n \frac{u_{i+1, j}^n - u_{i-1, j}^n}{2 \Delta y} + \frac{1}{Re} \left( \frac{u_{i, j-1}^n - 2 u_{ij}^n + u_{i, j+1}^n}{\Delta x^2} + \frac{u_{i-1, j}^n - 2 u_{ij}^n + u_{i+1, j}^n}{\Delta y^2} \right) \\
    \frac{v_{ij}^\star - v_{ij}^n}{\Delta t} = - u_{ij}^n  \frac{v_{i, j+1}^n - v_{i, j-1}^n}{2 \Delta x} - v_{ij}^n \frac{v_{i+1, j}^n - v_{i-1, j}^n}{2 \Delta y} + \frac{1}{Re} \left( \frac{v_{i, j-1}^n - 2 v_{ij}^n + v_{i, j+1}^n}{\Delta x^2} + \frac{v_{i-1, j}^n - 2 v_{ij}^n + v_{i+1, j}^n}{\Delta y^2} \right) \\
\end{eqnarray}
$$

2. Solve the Poisson equation for the pressure field $p$.

$$
\begin{equation}
    \frac{p_{i, j-1} - 2 p_{ij} + p_{i, j+1}}{\Delta x^2} + \frac{p_{i-1, j} - 2 p_{ij} + p_{i+1, j}}{\Delta y^2} = \frac{1}{\Delta t} \left( \frac{v_{i, j+1}^\star - v_{i, j-1}^\star}{2 \Delta x} + \frac{u_{i+1, j}^\star - u_{i-1, j}^\star}{2 \Delta y} \right)
\end{equation}
$$

3. Update the intermediate velocity with the pressure gradient to get a divergence-free velocity field.

$$
\begin{eqnarray}
    u_{ij}^{n+1} = u_{ij}^\star - \Delta t \; \frac{p_{i, j+1} - p_{i, j-1}}{2 \Delta x} \\
    v_{ij}^{n+1} = v_{ij}^\star - \Delta t \; \frac{p_{i+1, j} - p_{i-1, j}}{2 \Delta y}
\end{eqnarray}
$$

### Your assignment

You will solve discrete equations in a square domain $\left[ 0, 1 \right] \times \left[ 0, 1 \right]$ using a uniform grid with the same grid spacing in $x$ and $y$ directions ($n_x = n_y = 41$, $\Delta x = \Delta y$).

The Reynolds number is set to $Re = 10$.

Initial conditions:

$$
\begin{eqnarray}
    u \left( t = 0 \right) &=&
    \begin{cases}
        1 \quad \text{at} \; y = 1 \\
        0 \quad \text{everywhere else}
    \end{cases} \\
    v \left( t = 0 \right) &=& 0 \\
    p \left( t = 0 \right) &=& 0
\end{eqnarray}
$$

Boundary conditions:

* At $x=0$: $u = 0$, $v = 0$, and $\frac{\partial p}{\partial x} = 0$
* At $x=1$: $u = 0$, $v = 0$, and $\frac{\partial p}{\partial x} = 0$
* At $y=0$: $u = 0$, $v = 0$, and $\frac{\partial p}{\partial y} = 0$
* At $y=1$: $u = 1$, $v = 0$, and $p = 0$

Compute $n_t = 9000$ time steps with a time-step size $\Delta t = 10^{-4}$.

To solve the discrete pressure Poisson equation, you will implement and use the method of conjugate gradients that we implemented in class.
If you wish, you can even use Numba to accelerate the time to convergence of the Poisson equation.

You should stop the conjugate-gradient solver once the normalised $L_2$-norm of the difference is smaller or equal than $10^{-6}$, i.e. when:

$$
\begin{equation}
    \frac{\| p^{k+1} - p^k \|_{L_2}}{n_{x}n_{y}} \leq 10^{-6}
\end{equation}
$$

<img src="grid_figure.png" width="700">

**Note** : To follow the above figure as well as the indexing in the above discretized equations, we firstly need to decide on the axes-orientation and secondly to meet the Python axes-direction. In order to keep physical orientation in our computational grid (again that's just a personal preference) we initialise our variable as **[ny, nx]** and in order to meet Python axes-direction we flip the top (north) and bottom (south) BC, by maintaining the rest as they are.

## Implement your solution (50 points)

Implement your solution in this section.
You can use as many code cells as you want.

In [1]:
#-------- Import libraries --------
import numpy as np
import matplotlib.pyplot as pyp

In [ ]:
def du_dt_convective(ux, uy, du_dx, du_dy):
    return ux*du_dx + uy*du_dy

def du_dt_viscous(Re, d2u_dx2, d2u_dy2):
    return 1 / Re * (d2u_dx2 + d2u_dy2):
    


In [3]:
#-------- Write code --------

class VelocityField(self):
    


    

## Assessment (50 points)

Answer questions in this section.

Do not try to delete or modify the cells that are already present.
For each question, make sure to provide your answer in the cell **just above** the empty cell.

### Part 1 (15 points)

Plot the filled contour of the pressure scalar field and the streamlines of the velocity vector field after $9000$ time steps.

Here is a figure of what your solution should look like:

![figure1](cavity.png)

**Figure 1:** Filled contour of the pressure field and streamlines of the velocity field in a two-dimensional lid-driven cavity at Reynolds number $Re = 10$.

**Hint:** For the filled contour, I used the function [`pyplot.contourf()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.contourf.html) with $51$ levels between $-1$ and $1$. For the velocity streamlines, I used the function [`pyplot.streamplot`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.streamplot.html#matplotlib.axes.Axes.streamplot).

In [4]:
#-------- Write code --------


## Part 2 (20 points)

Compute the divergence of the velocity field after $9000$ time steps and plot the filled contour of the absolute value.

In two dimensional the divergence of the velocity vector field is:

$$
\begin{equation}
    \nabla \cdot \mathbf{u} = \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}
\end{equation}
$$

To compute the divergence, you will use the following discrete version:

$$
\begin{equation}
    \left( \nabla \cdot \mathbf{u} \right)_{i, j} =  \frac{u_{i, j+1} - u_{i, j-1}}{2 \Delta x} + \frac{v_{i+1, j} - v_{i-1, j}}{2 \Delta y}
\end{equation}
$$

(Note that using the discrete equation above, the divergence will be computed only at interior points, not on the boundaries.)

Here is a figure of what your solution should look like:

![figure2](divergence.png)

**Figure 2:** Filled contour of the absolute divergence of the velocity field in a two-dimensional lid-driven cavity at Reynolds number $Re = 10$. ($51$ levels between $0$ and $1$.)

In [5]:
#-------- Write code --------


## Part 3 (15 points)

What's the $L_2$-norm of the divergence $\nabla \cdot \mathbf{u}$ (after $9000$ time steps) divided by the number of interior points?

$$
\begin{equation}
    \frac{\| \nabla \cdot \mathbf{u} \|_{L_2}}{\left( n_x - 2 \right) \left( n_y - 2 \right)}
\end{equation}
$$

**Hint:** If everything goes smooth, you should obtain a _small_ value (I got a value of about $2.94 \times 10^{-3}$).

**Important:** Print your result.

In [6]:
#-------- Write code --------
